## SVM et SVM multi classes

In [76]:
import numpy as np
import random
import pandas as pd
import qpsolvers
from qpsolvers import solve_qp
import cvxpy

Soit $X$ la matrice représentant l'ensemble des données
avec $ X \in \mathbb{R^{n*d}} $

Le but d'un SVM est de trouver l'hyperplan qui sépare "le mieux" les données reparties en deux classes.

Nous allons tout d'abord implémenter un SVM linéaire classique, puis nous nous pencherons sur la construction d'un SVM multi classe.

## SVM linéaire pour k = 2

On cherche l'hyperplan séparant au mieux les données.

Soit un jeu de données $X \in \mathbb{R^{nxd}}$ et y son vecteur d'étiquettes.
On suppose qu'on dispose de deux labels $\{-1,1\}$


L'hyperplan permettant de séparer les données est caracterisé par $w \in \mathbb{R^d}$ et $b \in \mathbb{R}$

On cherche à construire un classifieur h tel que:

\begin{equation}
    h(x)=
    \begin{cases}
      1, & \text{si}\ w.x + b \geq 0 \\
      -1, & \text{sinon}
    \end{cases}
  \end{equation} 

Soit $(x_i, y_i)$ un individu du jeu de données et son label associé.
alors \begin{equation}
    y_i=
    \begin{cases}
      1, & \text{si}\ w.x_i + b \geq 1 \\
      -1, & \text{si}\ w.x_i + b \leq -1 
    \end{cases}
  \end{equation}
  
On a donc $\forall i \in [n]\  y_i(w.x_i + b) \geq 1$.

Les points qui satisfont l'égalité sont sur la marge autour de l'hyperplan.
Trouver un hyperplan qui sépare "le mieux" les données revient à maximiser la taille de cette marge.

Soit deux points $x_+$ et $x_-$ de part et d'autre de cette marge.

Alors la taille de la marge est $m = (x_+ - x_-).\dfrac{w}{||w||}$

Comme $x_+.w + b = 1$ et $x_-.w + b = -1$

On a, $m = \dfrac {2}{||w||}$

On cherche donc à résoudre le programme suivant:

$$min\  ||w||$$ 

$$s.c   \ \ \  y_i(w.x_i + b) \geq 1 \ \ \ \ \ \ \forall i \in [n]$$

On peut alors se ramener à un problème quadratique 
(car $||w||  = w^T.w$) et résoudre son dual.

In [73]:
class SVMClassifier(object):
    
    def __init__(self):
        self.w = None
        self.b = None
        self.label1 = None
        self.label2 = None
    
    def train(self, X_train, y_train):
        n = X_train.shape[0]
        self.label1 = np.unique(y_train.values)[0] 
        self.label2 = np.unique(y_train.values)[1]
        
        # Variables permettant de caractériser le problème dual
        # Ici P est toujours semi-definie positive
        X = X_train.values
        y = np.array([1 if x == self.label1 else -1 for x in y_train])
        y_diag = np.diag(y)
        P = y_diag@X@X.T@y_diag
        q=-1*np.ones(n)
        G=np.vstack((y,-1*y,-1*np.eye(n)))
        h=np.zeros(n+2)
        
        # Résolution du problème dual
        solDual= solve_qp(P, q, G, h, solver = 'cvxpy')
        
        # Récupération de w et b
        self.w = X.T@y_diag@solDual
        support_indices=np.where(solDual>0.001)[0]
        self.b = 1/y[support_indices[0]]-self.w.T@X[support_indices[0]]
        
        return 1
    
    def predict(self, X_test):
        n = X_test.shape[0]
        return [self.label1 if np.dot(self.w, x) + self.b >= 0 else self.label2 for x in X_test]

Le SVM multi classe que nous allons implémenter utilise une approche "One to One".
Pour chaque paire de classes distinctes, nous allons construire un classifieur SVM.
Ainsi, nous disposerons de k(k-1)/2 classifieurs "one to one".
Nous utiliserons ensuite un vote à majorité pour classifier.

In [74]:
class MultClassSVMClassifier(object):
    
    def __init__(self):
        self.classifiers = []
    
    def train(self, X_train, y_train):
        # Récupération du nombre de classes
        k = np.unique(y_train.values).size
        labels = np.unique(y_train.values)
        # Création des k(k-1)/2 classifieurs
        for i in range(k):
            for j in range(i+1, k):
                svm = SVMClassifier()
                svm.train(X_train.loc[(y_train == labels[i]) | (y_train == labels[j])], y_train.loc[(y_train == labels[i])| (y_train == labels[j])])
                self.classifiers.append(svm)
                
    def predict(self, X_test):
        predicts = []
        for classifier in self.classifiers:
            predicts.append(classifier.predict(X_test))
            
        predicts = np.transpose(predicts)
        # Vote à majorité 
        result = [np.bincount(x).argmax() for x in predicts]
        return result
        